In [1]:
strt = '2004-10-01'
nd = '2009-09-30'
cellsize=30

plapse = 0.00004 # m/m
tmaxlapse = -1.241379e-02# deg C / m
tminlapse = -1.103448e-02# deg C / m
tavelapse = -0.007651# deg C / m

In [2]:
site_elev = 3018 # met station site elevation in m

In [3]:
tmp = pd.read_csv('./data/nwt_hrly_met_data_20160527/data/c1_hrly_met_data_master_QC_infill.csv')
tmp.index = pd.DatetimeIndex(tmp.datetime)

In [4]:
c1 = pd.DataFrame()
c1['Tmin'] = tmp.temp_FILL2.resample('D').min() # min temperature [degC]
c1['Tmax'] = tmp.temp_FILL2.resample('D').max() # max temperature [degC]
c1['Tave'] = tmp.temp_FILL2.resample('D').mean() # mean temperature [degC]
c1['P'] = tmp.ppt_h.resample('D').sum()/1000. # total precip. [m]

In [5]:
c1 = c1[strt:nd]

In [6]:
import gdal

In [7]:
fl = '/RHESSys/Como/auxdata/gridmet_dem_%sm.tiff'%cellsize
ds = gdal.Open(fl)
dem = np.array(ds.GetRasterBand(1).ReadAsArray())

In [8]:
fl = '/RHESSys/Como/auxdata/gridmet_index_%sm.tiff'%cellsize
ds = gdal.Open(fl)
idx = np.array(ds.GetRasterBand(1).ReadAsArray())

In [9]:
m,k = idx.shape
n = m*k
idx.shape = m*k

In [10]:
dem.shape = m*k

In [11]:
header = '%s\n%s %s %s 1'%(n,c1.index[0].year,c1.index[0].month, c1.index[0].day)

In [12]:
header

'27270\n2004 10 1 1'

In [13]:
out = pd.DataFrame(np.ndarray((len(c1)+2,n)))

In [14]:
out.iloc[0,:] = idx # add the index numbers to the data frame
out.iloc[1,:] = dem # add the dem values to the data frame

In [15]:
# make copies of the data frame for population
Tmax = out.copy()
Tmin = out.copy()
Tave = out.copy()
P = out.copy()

In [16]:
elev_diff = dem - site_elev

ct = 2 # start at 2 b/c the first 2 rows are full
for day in c1.index:
    Tmax.iloc[ct,:] = c1.loc[day].Tmax + (tmaxlapse*elev_diff)
    Tmin.iloc[ct,:] = c1.loc[day].Tmin + (tminlapse*elev_diff)
    Tave.iloc[ct,:] = c1.loc[day].Tave + (tavelapse*elev_diff)
    P.iloc[ct,:] = c1.loc[day].P + (plapse*elev_diff)

Tmax = Tmax.as_matrix()
Tmin = Tmin.as_matrix()
Tave = Tave.as_matrix()
P = P.as_matrix()

In [17]:
P[P<0] = 0

In [20]:
np.savetxt('/RHESSys/Como/clim/grid%s.tmax'%(cellsize),Tmax,header=header,comments='',fmt='%3.2f')
np.savetxt('/RHESSys/Como/clim/grid%s.tmin'%(cellsize),Tmin,header=header,comments='',fmt='%3.2f')
np.savetxt('/RHESSys/Como/clim/grid%s.tavg'%(cellsize),Tave,header=header,comments='',fmt='%3.2f')
np.savetxt('/RHESSys/Como/clim/grid%s.rain'%(cellsize),P,header=header,comments='',fmt='%3.6f')

In [19]:
n

27270